# 02. Embeddings với Claude và LangChain

## Mục tiêu
- Hiểu vai trò của embeddings trong hệ thống AI
- Tạo embeddings cho queries và documents
- So sánh các embedding models khác nhau
- Hiểu về Anthropic/Claude embeddings

## 1. Embeddings là gì?

### Định nghĩa
Embeddings là cách biểu diễn văn bản dưới dạng vector số học trong không gian nhiều chiều. Mỗi văn bản được chuyển thành một vector, và các văn bản có ý nghĩa tương tự sẽ có vector gần nhau trong không gian.

### Vai trò của Embeddings
1. **Semantic Search**: Tìm kiếm dựa trên ý nghĩa thay vì từ khóa
2. **Similarity Matching**: So sánh độ tương đồng giữa các văn bản
3. **Clustering**: Nhóm các văn bản có nội dung liên quan
4. **RAG Systems**: Tìm kiếm thông tin liên quan từ knowledge base

## 2. Tình hình Anthropic/Claude Embeddings

### Hiện tại (2024)
- Anthropic chưa cung cấp public embedding API
- Claude models (Opus, Sonnet, Haiku) chỉ hỗ trợ text generation
- `langchain-anthropic` package chưa có embedding class

### Giải pháp thay thế
Khi làm việc với Claude, chúng ta có thể sử dụng:
1. **OpenAI Embeddings**: Chất lượng cao, dễ tích hợp
2. **HuggingFace Embeddings**: Miễn phí, nhiều model lựa chọn
3. **Cohere Embeddings**: Hiệu suất tốt cho multilingual
4. **Local Embeddings**: Sentence-transformers cho privacy

In [ ]:
# Cài đặt các packages cần thiết
!pip install langchain langchain-openai langchain-huggingface sentence-transformers numpy

## 3. Sử dụng OpenAI Embeddings

In [ ]:
from langchain_openai import OpenAIEmbeddings
import os
import numpy as np

# Khởi tạo OpenAI Embeddings
# Lưu ý: Cần có OPENAI_API_KEY trong environment
openai_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # hoặc text-embedding-3-large
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [ ]:
# Tạo embeddings cho một query
query = "Claude là gì và có những khả năng nào?"
query_embedding = openai_embeddings.embed_query(query)

print(f"Embedding dimension: {len(query_embedding)}")
print(f"Embedding vector (first 10 values): {query_embedding[:10]}")

In [ ]:
# Tạo embeddings cho nhiều documents
documents = [
    "Claude là một AI assistant được phát triển bởi Anthropic",
    "Claude có khả năng hiểu ngữ cảnh và trả lời câu hỏi phức tạp",
    "Python là ngôn ngữ lập trình phổ biến",
    "Machine Learning là một nhánh của AI"
]

doc_embeddings = openai_embeddings.embed_documents(documents)

print(f"Number of documents: {len(doc_embeddings)}")
print(f"Each embedding dimension: {len(doc_embeddings[0])}")

## 4. Sử dụng HuggingFace Embeddings (Miễn phí)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# Sử dụng model all-MiniLM-L6-v2 (nhẹ và hiệu quả)
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

In [ ]:
# Tạo embeddings với HuggingFace
hf_query_embedding = hf_embeddings.embed_query(query)
hf_doc_embeddings = hf_embeddings.embed_documents(documents)

print(f"HF Embedding dimension: {len(hf_query_embedding)}")
print(f"HF Number of doc embeddings: {len(hf_doc_embeddings)}")

## 5. Tính toán Similarity

In [ ]:
def cosine_similarity(vec1, vec2):
    """Tính cosine similarity giữa 2 vectors"""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    return dot_product / (norm1 * norm2)

# Tính similarity giữa query và các documents
print("Similarity scores với OpenAI embeddings:")
for i, doc in enumerate(documents):
    similarity = cosine_similarity(query_embedding, doc_embeddings[i])
    print(f"Document {i+1}: {similarity:.4f} - {doc[:50]}...")

In [ ]:
# So sánh với HuggingFace embeddings
print("\nSimilarity scores với HuggingFace embeddings:")
for i, doc in enumerate(documents):
    similarity = cosine_similarity(hf_query_embedding, hf_doc_embeddings[i])
    print(f"Document {i+1}: {similarity:.4f} - {doc[:50]}...")

## 6. Multilingual Embeddings

In [ ]:
# Sử dụng multilingual model
multilingual_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# Test với nhiều ngôn ngữ
multilingual_texts = [
    "Claude is an AI assistant",  # English
    "Claude là một trợ lý AI",     # Vietnamese
    "Claude est un assistant IA",  # French
    "Claudeは AIアシスタントです"    # Japanese
]

ml_embeddings = multilingual_embeddings.embed_documents(multilingual_texts)

# So sánh similarity giữa các ngôn ngữ
print("Cross-lingual similarity:")
for i in range(len(multilingual_texts)):
    for j in range(i+1, len(multilingual_texts)):
        sim = cosine_similarity(ml_embeddings[i], ml_embeddings[j])
        print(f"{multilingual_texts[i]} <-> {multilingual_texts[j]}: {sim:.4f}")

## 7. Best Practices cho Embeddings

### 1. Chọn Model phù hợp
- **OpenAI**: Chất lượng cao, tốn phí
- **HuggingFace**: Miễn phí, nhiều lựa chọn
- **Cohere**: Tốt cho multilingual

### 2. Preprocessing văn bản
- Loại bỏ ký tự đặc biệt không cần thiết
- Chuẩn hóa format (lowercase, spacing)
- Chunk documents hợp lý

### 3. Caching embeddings
- Lưu embeddings đã tính để tái sử dụng
- Sử dụng vector databases (Pinecone, Weaviate, ChromaDB)

In [ ]:
# Ví dụ: Caching embeddings với dict
embedding_cache = {}

def get_embedding_cached(text, embeddings_model):
    """Lấy embedding với caching"""
    if text in embedding_cache:
        print(f"Using cached embedding for: {text[:30]}...")
        return embedding_cache[text]
    
    print(f"Computing new embedding for: {text[:30]}...")
    embedding = embeddings_model.embed_query(text)
    embedding_cache[text] = embedding
    return embedding

# Test caching
text1 = "Claude là AI assistant"
emb1 = get_embedding_cached(text1, hf_embeddings)
emb2 = get_embedding_cached(text1, hf_embeddings)  # Sẽ dùng cache

## 8. Tích hợp Embeddings với Claude

Mặc dù Claude không có embedding API, chúng ta có thể kết hợp:
1. Dùng embedding model khác để search/retrieve
2. Dùng Claude để generate responses

Đây chính là pattern của RAG systems!

In [ ]:
# Ví dụ workflow RAG với Claude
from typing import List, Tuple

def simple_rag_with_claude(query: str, knowledge_base: List[str]) -> Tuple[List[str], str]:
    """
    Simple RAG workflow:
    1. Embed query và documents
    2. Tìm relevant documents
    3. Dùng Claude để generate response
    """
    # Bước 1: Tạo embeddings
    query_emb = hf_embeddings.embed_query(query)
    doc_embs = hf_embeddings.embed_documents(knowledge_base)
    
    # Bước 2: Tính similarity và rank
    similarities = []
    for i, doc_emb in enumerate(doc_embs):
        sim = cosine_similarity(query_emb, doc_emb)
        similarities.append((sim, i, knowledge_base[i]))
    
    # Sort by similarity (descending)
    similarities.sort(key=lambda x: x[0], reverse=True)
    
    # Lấy top 2 documents
    top_docs = [item[2] for item in similarities[:2]]
    
    # Bước 3: Tạo prompt cho Claude (giả lập)
    prompt = f"""
    Based on the following information:
    {' '.join(top_docs)}
    
    Answer the question: {query}
    """
    
    return top_docs, prompt

# Test
kb = [
    "Claude can analyze images and documents",
    "Claude was created by Anthropic with focus on safety",
    "Python is used for data science",
    "Claude can write and debug code"
]

relevant_docs, prompt = simple_rag_with_claude("What can Claude do?", kb)
print("Relevant documents found:")
for doc in relevant_docs:
    print(f"- {doc}")
print(f"\nPrompt for Claude:\n{prompt}")

## Tổng kết

1. **Embeddings** là nền tảng cho semantic search và RAG
2. **Anthropic/Claude** chưa có embedding API, nhưng có thể kết hợp với các embedding models khác
3. **OpenAI Embeddings** cho chất lượng tốt nhất nhưng tốn phí
4. **HuggingFace Embeddings** miễn phí và đủ tốt cho nhiều use cases
5. **Cosine similarity** là metric phổ biến để so sánh embeddings
6. **Caching** embeddings giúp tiết kiệm chi phí và thời gian

### Next Steps
- Tìm hiểu về Vector Databases
- Xây dựng RAG system hoàn chỉnh
- Optimize embedding search với indexing